In [1]:

import numpy as np
import pandas as pd
from tqdm import tqdm
import time
import iisignature as sig
from scipy.stats import pearsonr


from time import sleep


from sklearn.preprocessing import LabelEncoder,StandardScaler,Normalizer,MinMaxScaler

from sklearn.model_selection import GridSearchCV, KFold, cross_val_score, train_test_split
from sklearn.pipeline import Pipeline
from sklearn.metrics import roc_auc_score, accuracy_score, f1_score,r2_score,mean_squared_error

from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier,RandomForestRegressor,GradientBoostingRegressor
from sklearn.linear_model import LogisticRegression,Lasso
from sklearn.svm import SVC,SVR
from sklearn.neural_network import MLPRegressor
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF
from sklearn.neighbors import KNeighborsClassifier


from tslearn.svm import TimeSeriesSVC,TimeSeriesSVR
from tslearn.neighbors import KNeighborsTimeSeriesClassifier
from tslearn.preprocessing import TimeSeriesScalerMinMax
from tslearn.neural_network import TimeSeriesMLPRegressor

from Neural_Decoding.preprocessing_funcs import get_spikes_with_history

In [2]:
def ml_method_setup(method, X_train, y_train):
    
        if method == 'ts_knn':
                parameters = {'n_neighbors': [1, 3, 5, 7],'metric': ['euclidean', 'dtw']}
                clf = GridSearchCV(KNeighborsTimeSeriesClassifier(),
                                parameters, cv=5, n_jobs=-1, verbose=10)
                clf.fit(X_train, y_train)

        elif method == 'ts_svc':
                parameters = {'C': [0.1, 1, 10],'kernel': ['linear', 'rbf'],'gamma': ['scale', 'auto', 0.1]}
                clf = GridSearchCV(TimeSeriesSVC(random_state=0, probability=True),
                                parameters, cv=5, n_jobs=-1, verbose=10)
                clf.fit(X_train, y_train)
        
        elif method == 'r_ts_svr':
                parameters = {'C': [0.1, 1, 10],'epsilon': [0.01, 0.1, 0.5],  'kernel': ['linear', 'rbf']}
                clf = GridSearchCV(TimeSeriesSVR(),
                                parameters, cv=5, n_jobs=-1, verbose=10)
                clf.fit(X_train, y_train)

        elif method == 'r_ts_neuralnetwork':
                parameters = {'hidden_layer_sizes': [(50,), (100,), (50, 50)], 
                           'activation': ['relu', 'tanh'],  
                           'learning_rate': ['constant', 'adaptive'],  
                           'alpha': [0.0001, 0.001, 0.01]}
                clf = GridSearchCV(TimeSeriesMLPRegressor(),
                                parameters, cv=5, n_jobs=-1, verbose=10)
                clf.fit(X_train, y_train)

        elif method == 'logisticregression':

                lr = LogisticRegression(random_state=0)
                parameters = {'C': [0.1, 0.2, 0.5, 1, 2, 5, 10],
                                'solver': ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga']}
                clf = GridSearchCV(lr, parameters,cv =5, n_jobs=-1, verbose=10)
                clf.fit(X_train, y_train)

        elif method == 'svc':
                svc = SVC(random_state=0, probability=True)
                parameters = {'kernel': ['rbf', 'poly'], 'shrinking': [True, False],
                                'C': [0.1, 1, 10]}
                clf = GridSearchCV(svc, parameters,cv=5, n_jobs=-1, verbose=10)
                clf.fit(X_train, y_train)

        elif method == 'knnclassifier':
                knn = KNeighborsClassifier()
                parameters = {'n_neighbors': range(3, 30, 2), 'weights': ['uniform', 'distance']}
                clf = GridSearchCV(knn, parameters, cv=5, n_jobs=-1, verbose=10)
                clf.fit(X_train, y_train)

        elif method == 'adaboostclassifier':
                ada = AdaBoostClassifier(random_state=0)
                parameters = {'n_estimators': [50, 100], 'learning_rate': [0.5, 1, 2]}
                clf = GridSearchCV(ada, parameters, cv=5, n_jobs=-1, verbose=10)
                clf.fit(X_train, y_train)

        elif method == 'randomforestclassifier':
            rf = RandomForestClassifier(random_state=0)
            parameters = {'min_weight_fraction_leaf': [0.1, 0.5],
                            'bootstrap': [True, False],
                            'max_depth': (2, 5),
                            'max_leaf_nodes': (2, 5),
                            'n_estimators': (100, 200)}
            clf = GridSearchCV(rf, parameters, cv=5, n_jobs=-1, verbose=10)
            clf.fit(X_train, y_train)

        elif method == 'r_lassoregression':
                lr = Lasso()
                parameters = {'alpha': [0.1, 0.2, 0.5, 1, 2, 5, 10]}
                clf = GridSearchCV(lr, parameters, cv=5, n_jobs=-1, verbose=10)
                clf.fit(X_train, y_train)
        
        elif method == 'r_svr':
                svr = SVR()
                parameters = {'C': [0.1, 1.0, 10.0], 'kernel': ['linear', 'rbf'], 
                               'gamma': ['scale', 'auto']}
                clf = GridSearchCV(svr, parameters, cv=5, n_jobs=-1, verbose=10)
                clf.fit(X_train, y_train)

        elif method == 'r_randomforestregression':
                rf = RandomForestRegressor()
                parameters =  {'n_estimators': [50, 100, 200], 'max_depth': [None, 10, 20], 'min_samples_split': [2, 5, 10]}
                clf = GridSearchCV(rf, parameters, cv=5, n_jobs=-1, verbose=10)
                clf.fit(X_train, y_train)

        elif method == 'r_gradientboostingregression':
                gb = GradientBoostingRegressor()
                parameters= {'n_estimators': [50, 100, 200], 'learning_rate': [0.01, 0.1, 0.2], 'max_depth': [3, 5, 7]}
                clf = GridSearchCV(gb, parameters, cv=5, n_jobs=-1, verbose=10)
                clf.fit(X_train, y_train)

        elif method == 'r_neuralnetwork':
                nn = MLPRegressor()
                parameters = {'hidden_layer_sizes': [(50,), (100,), (50, 50)], 'activation': ['relu', 'tanh'], 'alpha': [0.0001, 0.001, 0.01]}
                clf = GridSearchCV(nn, parameters, cv=5, n_jobs=-1, verbose=10)
                clf.fit(X_train, y_train)
        
        elif method == 'r_GaussianNB':
                kernel = 1.0 * RBF(length_scale=1.0)
                model = GaussianProcessRegressor(kernel=kernel, n_restarts_optimizer=10, random_state=42)
                parameters = {'kernel__length_scale': [0.1, 1.0, 10.0],}        
                clf = GridSearchCV(model, parameters, cv=5, n_jobs=-1,verbose=10,scoring='neg_mean_squared_error')
                clf.fit(X_train, y_train)
        else:
                clf = None

        return clf

In [3]:
def time_augment(arr):
    return np.vstack((arr.T, np.linspace(0, 1, num=arr.shape[0]))).T

In [4]:
def standard_scale_process(X_train,X_test):
    # data: np.array (num_samples, num_timestamps, num_channels)
    data_mean = np.nanmean(X_train,axis=0)
    data_std = np.nanstd(X_train,axis=0)
    if 0 in data_std:
      print('error')
    X_train = (X_train-data_mean)/data_std
    X_test = (X_test-data_mean)/data_std
    return X_train,X_test

In [5]:
def data_bins(X_train,y_train,X_test,y_test,bins_before,bins_after,bins_current):

    # use bin decoding to create array with (num_samples, num_bins(time), num_features)
  if bins_before == 0:
    X_train =get_spikes_with_history(X_train,bins_before,bins_after,bins_current)[:-bins_after,:,:]
    X_test = get_spikes_with_history(X_test,bins_before,bins_after,bins_current)[:-bins_after,:,:]
    y_train = y_train[:-bins_after]
    y_test = y_test[:-bins_after]

  elif bins_after == 0:
    X_train =get_spikes_with_history(X_train,bins_before,bins_after,bins_current)[bins_before:,:,:]
    X_test = get_spikes_with_history(X_test,bins_before,bins_after,bins_current)[bins_before:,:,:]
    y_train = y_train[bins_before:]
    y_test = y_test[bins_before:]

  else:
    X_train =get_spikes_with_history(X_train,bins_before,bins_after,bins_current)[bins_before:-bins_after,:,:]
    X_test = get_spikes_with_history(X_test,bins_before,bins_after,bins_current)[bins_before:-bins_after,:,:]
    y_train = y_train[bins_before:-bins_after]
    y_test = y_test[bins_before:-bins_after]

  return X_train,y_train,X_test,y_test

In [6]:
def data_flatten(X_train, X_test):
    X_train = X_train.reshape(X_train.shape[0], -1)
    X_test = X_test.reshape(X_test.shape[0], -1)
    return X_train, X_test

In [7]:
def data_process(X_train, X_test, scale='minmax' , time_aug=False):
    
    # X_train shape: (n_samples, n_features, n_timestamps)
    # X_test shape: (n_samples, n_features, n_timestamps)

    # ============ Data Preprocessing ============ 
    ts_min_max_scaler = TimeSeriesScalerMinMax()

    # ============ minmax =============
    if scale == 'minmax':
        scaler = ts_min_max_scaler.fit(X_train)
        X_train = scaler.transform(X_train)
        X_test = scaler.transform(X_test)

    # ============ standard =============
    elif scale == 'standard':
        print(X_train.shape)
        X_train, X_test = standard_scale_process(X_train, X_test)
    
    else: 
        pass

    # ============ time aurment =============
    if time_aug:
        X_train = np.array(list(map(time_augment, X_train)))
        X_test = np.array(list(map(time_augment, X_test)))

    return X_train, X_test

In [8]:
def signature(X_train,X_test,sig_level=2):
    # hape of X_train: (n_samples,n_timestamps,n_features)
    X_train = sig.sig(X_train,sig_level)
    X_test = sig.sig(X_test,sig_level)

    return X_train,X_test


In [9]:
def best_bins(X_train,y_train,X_test,y_test,reduced=True):
    if reduced:
        bins_before = [10,50,100]
        bins_after  = [0,10]
        bins_current = [0,1]
    else:
        bins_before = [5,10,25,50,75,100,250,500]
        bins_after = [5,10,25,50,75,100,250,500]
        bins_current = [0,1]

    r2_best = 0
    bin_b_best = 0
    bin_a_best = 0
    bin_c_best = 0

    for bin_before in tqdm(bins_before):
        for bin_after in bins_after:
            for bin_current in bins_current:
                X_train_b,y_train_b,X_test_b,y_test_b = data_bins(X_train,y_train,X_test,y_test,bin_before,bin_after,bin_current)
                X_train_flat, X_test_flat = data_flatten(X_train_b, X_test_b)
                kf = KFold(n_splits=3)
                
                for train_index, test_index in kf.split(X_train_flat):
                    X_train_cv, X_test_cv = X_train_flat[train_index], X_train_flat[test_index]
                    y_train_cv, y_test_cv = y_train_b[train_index], y_train_b[test_index]
                    clf = SVR()
                    clf.fit(X_train_cv,y_train_cv)
                    y_pred = clf.predict(X_test_cv)
                    r2 = r2_score(y_test_cv, y_pred)
                    if r2 > r2_best:
                        bin_b_best = bin_before
                        bin_a_best = bin_after
                        bin_c_best = bin_current
                        r2_best = r2

    return bin_b_best,bin_a_best,bin_c_best


In [10]:
def auto(X_train, X_test, y_train, y_test, 
             dataset, method, sig_level, 
             scale, time_aug,
             bins_before, bins_after, bins_current,
             file_path = '.'
             ):
    
    

    file_name = f"{dataset}_{method}"
    if scale == 'minmax':
        file_name += "_minmax"
    elif scale == 'standard':
        file_name += "_standard"
    else:
        file_name += "_none"

    if time_aug:
        file_name += "_timeaug"

    if method.startswith('r_'):
        
        
        X_train,y_train,X_test,y_test = data_bins(X_train,y_train,X_test,y_test,bins_before,bins_after,bins_current)
        X_train,X_test = data_process(X_train, X_test, scale, time_aug)

        if method.startswith('r_ts'):
            start_time = time.time()
            clf = ml_method_setup(method, X_train, y_train)
            y_pred = clf.predict(X_test)
            r2_ts = r2_score(y_test, y_pred)
            r_value_ts, p_value_ts = pearsonr(y_test, y_pred)
            end_time = time.time()
            time_ts = end_time - start_time

            score_ts = pd.DataFrame({'r2':r2_ts, 'rvalue':r_value_ts, 'time':time_ts}, index=[0])
            score_ts.to_csv(f"{file_path}{file_name}_ts.csv", index=False)

        else:
            # ==== signature ====
            start_time = time.time()
            X_train_sig, X_test_sig = signature(X_train, X_test, sig_level)
            clf_sig = ml_method_setup(method, X_train_sig, y_train)
            y_pred_sig = clf_sig.predict(X_test_sig)
            r2_sig = r2_score(y_test, y_pred_sig)
            r_value_sig, p_value_sig = pearsonr(y_test, y_pred_sig)
            end_time = time.time()
            time_sig = end_time - start_time

            score_sig = pd.DataFrame({'r2':r2_sig, 'rvalue':r_value_sig, 'time':time_sig}, index=[0])
            score_sig.to_csv(f"{file_path}{file_name}_sig.csv", index=False)
            
            

            # ==== flatten =====
            start_time = time.time()
            X_train_flat, X_test_flat = data_flatten(X_train, X_test)
            clf_flat = ml_method_setup(method, X_train_flat, y_train)
            y_pred_flat = clf_flat.predict(X_test_flat)
            r2_flat = r2_score(y_test, y_pred_flat)
            r_value_flat, p_value_flat = pearsonr(y_test, y_pred_flat)
            end_time = time.time()
            time_flat = end_time - start_time

            score_flat = pd.DataFrame({'r2':r2_flat, 'rvalue':r_value_flat, 'time':time_flat}, index=[0])
            score_flat.to_csv(f"{file_path}{file_name}_flat.csv", index=False)

    else:
        X_train, X_test = data_process(X_train, X_test, scale, time_aug)
        print(X_train.shape)
        
        if method.startswith('ts'):
            start_time = time.time()
            clf = ml_method_setup(method, X_train, y_train)
            y_pred = clf.predict(X_test)
            accuracy = accuracy_score(y_test, y_pred)
            end_time = time.time()
            time_ts = end_time - start_time

            score_ts = pd.DataFrame({'accuracy_score':accuracy, 'time':time_ts}, index=[0])
            score_ts.to_csv(f"{file_path}{file_name}_ts.csv", index=False)

        else:
            # ====== signature ======
            start_time = time.time()
            X_train_sig, X_test_sig = signature(X_train, X_test, sig_level)
            clf_sig = ml_method_setup(method, X_train_sig, y_train)
            y_pred_sig = clf_sig.predict(X_test_sig)
            accuracy_score_sig = accuracy_score(y_test, y_pred_sig)
            end_time = time.time()
            time_sig = end_time - start_time

            score_sig = pd.DataFrame({'accuracy_score':accuracy_score_sig, 'time':time_sig}, index=[0])
            score_sig.to_csv(f"{file_path}{file_name}_sig.csv", index=False)

            # ====== flatten ======
            start_time = time.time()
            X_train_flat, X_test_flat = data_flatten(X_train, X_test)
            clf_flat = ml_method_setup(method, X_train_flat, y_train)
            y_pred_flat = clf_flat.predict(X_test_flat)
            accuracy_score_flat = accuracy_score(y_test, y_pred_flat)
            end_time = time.time()
            time_flat = end_time - start_time
            
            score_flat = pd.DataFrame({'accuracy_score':accuracy_score_flat, 'time':time_flat}, index=[0])
            score_flat.to_csv(f"{file_path}{file_name}_flat.csv", index=False)
